In [1]:
import pandas as pds
from astropy.time import Time
from datetime import timedelta

# Get official leap seconds list from International Time Bureau

In [2]:
leap_seconds = pds.read_csv("https://www.ietf.org/timezones/data/leap-seconds.list", comment="#", sep='\t', names=["epoch","offset"], index_col=False) 

# Print leap seconds list 

In [19]:
for sec in leap_seconds.values: 
    print(Time("1900-01-01", scale='utc') + timedelta(seconds=float(sec[0]+sec[1]))) 

1972-01-01 00:00:00.000
1972-07-01 00:00:00.000
1973-01-01 00:00:00.000
1974-01-01 00:00:00.000
1975-01-01 00:00:00.000
1976-01-01 00:00:00.000
1977-01-01 00:00:00.000
1978-01-01 00:00:00.000
1979-01-01 00:00:00.000
1980-01-01 00:00:00.000
1981-07-01 00:00:00.000
1982-07-01 00:00:00.000
1983-07-01 00:00:00.000
1985-07-01 00:00:00.000
1988-01-01 00:00:00.000
1990-01-01 00:00:00.000
1991-01-01 00:00:00.000
1992-07-01 00:00:00.000
1993-07-01 00:00:00.000
1994-07-01 00:00:00.000
1996-01-01 00:00:00.000
1997-07-01 00:00:00.000
1999-01-01 00:00:00.000
2006-01-01 00:00:00.000
2009-01-01 00:00:00.000
2012-07-01 00:00:00.000
2015-07-01 00:00:00.000
2017-01-01 00:00:00.000


# generate C++ code for CDF++ lib

In [20]:

def build_leap_table():
    res = """constexpr std::array leap_seconds_tt2000 = 
    {"""
    for sec in leap_seconds.values: 
        date = Time("1900-01-01", scale='utc') + timedelta(seconds=float(sec[0]+sec[1]))
        res += f"// {date}\n"
        res += f"    std::pair{{ tt2000_t{{ ({sec[0]} - constants::seconds_1900_to_J2000)*1000*1000*1000 }} , {sec[1]} - 32 }},\n"
    res += "};\n"
    return res

def build_reverse_leap_table():
    res = """constexpr std::array leap_seconds_tt2000_reverse = 
    {"""
    for sec in leap_seconds.values: 
        date = Time("1900-01-01", scale='utc') + timedelta(seconds=float(sec[0]+sec[1]))
        res += f"// {date}\n"
        res += f"    std::pair{{ tt2000_t{{ ({sec[0]} - constants::seconds_1900_to_J2000 + {sec[1]} -32) *1000*1000*1000 }}  , {sec[1]} - 32 }},\n"
    res +="""};\n"""
    return res

with open('../include/cdf-leap-seconds.h','w') as leap_file:
    leap_file.write("""
#pragma once
/*------------------------------------------------------------------------------
-- This file is a part of the CDFpp library
-- Copyright (C) 2020, Plasma Physics Laboratory - CNRS
--
-- This program is free software; you can redistribute it and/or modify
-- it under the terms of the GNU General Public License as published by
-- the Free Software Foundation; either version 2 of the License, or
-- (at your option) any later version.
--
-- This program is distributed in the hope that it will be useful,
-- but WITHOUT ANY WARRANTY; without even the implied warranty of
-- MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
-- GNU General Public License for more details.
--
-- You should have received a copy of the GNU General Public License
-- along with this program; if not, write to the Free Software
-- Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
-------------------------------------------------------------------------------*/
/*-- Author : Alexis Jeandet
-- Mail : alexis.jeandet@member.fsf.org
----------------------------------------------------------------------------*/
#include "date/date.h"
#include <chrono>
#include <utility>
#include <array>
#include "cdf-chrono-constants.hpp"
namespace cdf::chrono::leap_seconds
{    
""")
    leap_file.write(build_leap_table())
    leap_file.write(build_reverse_leap_table())
    leap_file.write("""
} //namespace cdf
    """)

In [5]:
print("""constexpr std::array leap_seconds_J2000_reverse = 
{""")
for sec in leap_seconds.values: 
    date = str(Time("1900-01-01") + timedelta(seconds=float(sec[0]+sec[1]))).split()[0].split('-')
    print(f"    std::pair{{ duration_cast<seconds>( (sys_days {{ {date[0]}_y / {date[1]} / {date[2]} }} + 0h ) - _J2000).count() + {sec[1]} - 32 , {sec[1]} - 32 }},") 
print("""};""")


constexpr std::array leap_seconds_J2000_reverse = 
{
    std::pair{ duration_cast<seconds>( (sys_days { 1972_y / 01 / 01 } + 0h ) - _J2000).count() + 10 - 32 , 10 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1972_y / 07 / 01 } + 0h ) - _J2000).count() + 11 - 32 , 11 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1973_y / 01 / 01 } + 0h ) - _J2000).count() + 12 - 32 , 12 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1974_y / 01 / 01 } + 0h ) - _J2000).count() + 13 - 32 , 13 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1975_y / 01 / 01 } + 0h ) - _J2000).count() + 14 - 32 , 14 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1976_y / 01 / 01 } + 0h ) - _J2000).count() + 15 - 32 , 15 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1977_y / 01 / 01 } + 0h ) - _J2000).count() + 16 - 32 , 16 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1978_y / 01 / 01 } + 0h ) - _J2000).count() + 17 - 32 , 17 - 32 },
   

In [7]:
leap_seconds.values


array([[2272060800,         10],
       [2287785600,         11],
       [2303683200,         12],
       [2335219200,         13],
       [2366755200,         14],
       [2398291200,         15],
       [2429913600,         16],
       [2461449600,         17],
       [2492985600,         18],
       [2524521600,         19],
       [2571782400,         20],
       [2603318400,         21],
       [2634854400,         22],
       [2698012800,         23],
       [2776982400,         24],
       [2840140800,         25],
       [2871676800,         26],
       [2918937600,         27],
       [2950473600,         28],
       [2982009600,         29],
       [3029443200,         30],
       [3076704000,         31],
       [3124137600,         32],
       [3345062400,         33],
       [3439756800,         34],
       [3550089600,         35],
       [3644697600,         36],
       [3692217600,         37]])